# Financial Market Data Analysis

In this project i will conduct a time series analysis of historical financial market data on the S&P 500 ETF. The analysis is meant to be exploratory in nature, and is not meant to provide advice/recommendations.

## Getting Data

I will be using pandas: datareader to read in data from Yahoo Finance. The data will span 3 years, from 1st Jan 2017 to 1st Jan 2020.

### Import relevant packages

In [ ]:
pip install --upgrade mplfinance

In [ ]:
from pandas_datareader import data, wb
import pandas as pd
import numpy as np
import datetime
import seaborn as sns
import matplotlib.pyplot as plt
from mplfinance.original_flavor import candlestick_ohlc
import matplotlib.dates as mdates
from scipy.stats import norm
%matplotlib inline


In [ ]:
start=datetime.datetime(2017,1,1)
end=datetime.datetime(2020,1,1)

In [ ]:
SPY=data.DataReader('SPY','yahoo',start,end)

## Examine the data

In [ ]:
SPY.head()

In [ ]:
SPY.dtypes

In [ ]:
SPY.describe()

In [ ]:
np.sum(pd.isnull(SPY))

We are working with 754 observations over the time period, with no missing values. All our columns are of the float type.

## Visualize the data


In [ ]:
#Set font size
sns.set(font_scale=1.2)

# Set graph style
sns.set_style({"axes.facecolor": "1.0", "axes.edgecolor": "1.0",  'axes.grid': True, "grid.color": "0.85",
              "grid.linestyle": "-", 'axes.labelcolor': '0.2', "xtick.color": "0.3",
              'ytick.color': '0.3'})


plt.figure(figsize=(12,8))
plt.plot(SPY[['High', 'Low', 'Open','Adj Close']])
plt.title('Plot of Daily High, Low, Open and Adjusted Close prices')
plt.xlabel('Date',fontsize=20)
plt.ylabel('Price ($)',fontsize=20)
plt.legend(['High','Low','Open','Adj Close'])


In [ ]:
#Set font size
sns.set(font_scale=1.2)

# Set graph style
sns.set_style({"axes.facecolor": "1.0", "axes.edgecolor": "0.85",  'axes.grid': True, "grid.color": "0.85",
               "grid.linestyle": "-", 'axes.labelcolor': '0.4', "xtick.color": "0.4",
               'ytick.color': '0.4'})


plt.figure(figsize=(12,8))
plt.plot(SPY[['Volume']])
plt.title('Plot of Daily Volume Traded')
plt.xlabel('Date',fontsize=20)
plt.ylabel('Volume',fontsize=20)

Inspecting the plot on Daily Prices is difficult, given the overlaps between High, Low and Open prices. To better visualize the price changes, I will create a candlestick chart. Individual candlesticks display the size of price movements, with green corresponding to an upward movement and red representing a downward movement.

In order to use the candlestick_ohlc() function, I will need to format the dataframe accordingly. I will also limit the date range to 2 weeks initially, to better visualize the price changes.

In [ ]:
spy_candle=SPY.drop(['Volume','Adj Close'],axis=1)
dates = spy_candle.index.tolist()
dates = pd.DataFrame(mdates.date2num(dates), columns = ["Date"], index = spy_candle.index)
spy_candle=pd.concat([dates,spy_candle],axis=1)

In [ ]:
#Shorten time interval
new_start=datetime.date(2018,10,10)
new_end=datetime.date(2018,12,10)
ohlc = spy_candle[['Date','Open','High','Low','Close']]

#Create figure
fig ,ax = plt.subplots(figsize=(12,8))

#Plot candlestick
candlestick_ohlc(ax,ohlc.values,
                 width=0.6,
                colorup='green',
                colordown='red')
plt.xlabel("Date")
plt.ylabel("Price($)")

ax.set_xlim([new_start,new_end])
ax.set_ylim([255,290])
ax.set_axisbelow(True)
ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d'))

plt.tight_layout()


## Examine Returns

Use pct_change on 'Close' column to calculate returns

In [ ]:
SPY['Daily_Returns']=SPY['Close'].pct_change()
SPY.head()

Identify the dates where S&P 500 ETF had the worst and best returns over the period 1st Jan 2017 to 1st Jan 2020.

In [ ]:
SPY['Daily_Returns'].idxmin()


In [ ]:
SPY['Daily_Returns'].idxmax()

2018 was the year that S&P 500 ETFs had the best and worst returns during our observed time period.

## Visualize returns

Create a distribution plot for returns of S&P 500 ETF

In [ ]:
sns.displot(data=SPY['Daily_Returns'],color='blue',bins=100,height=8,aspect=12/8,kde=True)


## Moving Averages

Analyze moving averages of S&P 500 ETF in the year 2018. 
Plot the rolling 30 day average against close price.

In [ ]:
plt.figure(figsize=(12,8))
SPY['Close'].loc['2018-01-01':'2019-01-01'].rolling(window=30).mean().plot(label='30-Day Average')
SPY['Close'].loc['2018-01-01':'2019-01-01'].plot(label='S&P Close')

## Value at Risk
<br>
VaR is an attempt to summarize the total risk in a portfolio with a single number. <br> This helps financial institutions assess whether they have sufficient capital reserves set aside to cover losses.
It is defined as the worst loss expected from holding a portfolio over a given period of time, given a confidence level 1 - 
$\alpha$ <br>
A more simplified way of looking at it: Given $\alpha$ of 1 %, and a duration of 1 month, what is the maximum loss over that month? <br>
<br>
### Examine 2 definitions of VaR<br>
1. VaR = expected profit/loss minus worst case loss at the 1-$\alpha$ confidence level
2. Absolute VaR = - worst case loss at the 1-$\alpha$ confidence level

### 2 ways of calculating VaR<br>
1. Parametric approach
2. Historical, or non-parametric approach


#### Notes
1. Use Adj Close for calculating returns instead, as it gives a better idea of the overall value of the stock.
2. Assume that a company bought 50,000 shares of S&P ETFs on 1st Jan 2020, at $312 per share.
3. Find the one-day 99% VaR


#### 1. Parametric approach
Rank the daily return rates from the smallest to largest, and use $\alpha$ % to estimate the VaR of an investment.

In [ ]:
SPY_var=SPY.drop('Daily_Returns',axis=1)

In [ ]:
SPY_var['Daily_Returns']=SPY_var['Adj Close'].pct_change(periods=-1)
SPY_var=SPY_var[:-1]

In [ ]:
Num_shares=50000
Price_shares=312

para_VaR=Num_shares*Price_shares*SPY_var['Daily_Returns'].std()*norm.ppf(0.99,0,1)
print('The one-day 99% VaR using parametric approach is {}'.format(round(para_VaR,2)))

2. Historical Approach

In [ ]:
AbsVaR=np.quantile((SPY_var['Daily_Returns'].tolist()),0.01)*-1*Num_shares*Price_shares

In [ ]:
meanVaR=np.mean(SPY_var['Daily_Returns'].tolist())*Num_shares*Price_shares

In [ ]:
hist_VaR=meanVaR+AbsVaR
print('The one-day 99% VaR using historical approach is {}'.format(round(hist_VaR,2)))